# Pre-processing solver dataset and output clean dataset which is the input for clustering.

## Import packages

In [1]:
import pandas as pd
import regex as re
import pickle
import nltk

#from thoth.lab import common
from thoth.lab import solver
from nltk.tokenize import TreebankWordTokenizer
from string import punctuation  
from IPython.display import display, Markdown
from tqdm import tqdm

In [2]:
pd.set_option('max_colwidth', 1000)

## Set environment variables to retrieve data from Ceph and map to dataframe 
### DO NOT Run this everytime. 

In [3]:
import os
os.environ['THOTH_S3_ENDPOINT_URL'] = 'https://s3.upshift.redhat.com/'
os.environ['THOTH_CEPH_BUCKET'] = 'thoth'
os.environ['THOTH_CEPH_HOST'] = 'https://s3.upshift.redhat.com/'
os.environ['THOTH_CEPH_BUCKET_PREFIX'] = 'data/thoth'
os.environ['THOTH_DEPLOYMENT_NAME'] = 'thoth-psi-stage'

Connect to thoth storage

In [ ]:
from thoth.storages import SolverResultsStore
store = SolverResultsStore(region="us-east-1")
store.connect()

### All the result in solver report are described below:

- **environment,** information about the environment on which the package has being solved;
- **environment_packages,** information about external packages installed on the environment;
- **errors,** if the installation of a package was not succesfull there will be information stored for each package error;
    - **details,**
        - **command**,
        - **message**,
        - **return_code**,
        - **stderr**,
        - **stdout**,
        - **timeout**,
    - **index_url,** from where the package was download;
    - **package_name;**
    - **package_version;**
    - **is_provided_package,** flag for storing package;
    - **is_provided_package_version,** flag for storing package;
    - **type,** error type;
- **tree**, all the packages installed in the dependency tree and information about them;
    - **dependencies**
    - **metadata** of the package as taken from importlib_metadata;
    - **index_url** from where the package was download;
    - **package_name;**
    - **package_version;**
    - **sha256;**
- **unparsed** if there are packages in the tree that could not be parsed;
- **unresolved,** if there are packages in the tree that could not be solved;

### Currently grafana metrics from the SolverResultsStore show over 700k results. Each solver has around 20k - 30k results with solver error. 

This block filters for documents with solver errors(Retrieve solver reports from Ceph and save only if it has "errors" in it)


In [ ]:
solver_reports_extracted_data = []
solver_errors = []

for document_id in store.get_document_listing():
    try:
        solver_document = store.retrieve_document(document_id=document_id)
        solver_report_extracted_data = solver.extract_data_from_solver_metadata(solver_document["metadata"])
        errors = solver.extract_errors_from_solver_result(solver_document["result"]["errors"])   
        for error in errors:
            error['document_id'] = solver_report_extracted_data['document_id']
            error['datetime'] = solver_report_extracted_data['datetime']
            error['analyzer_version'] = solver_report_extracted_data['analyzer_version']
            error['environment'] = solver_document["result"]["environment"]
            solver_errors.append(error)
    except Exception as e:
        print(document_id, e)

In [ ]:
solver_total = pd.DataFrame(solver_errors).reset_index()
solver_total.to_csv('error_data.csv', index=False)
len(solver_total)

## Load Error data from csv file created in the above step

In [4]:
solver_total_errors_df = pd.read_csv('error_data.csv')

In [5]:
solver_total_errors_df.head(5)

,index,package_name,package_version,index_url,type,command,message,return_code,stderr,stdout,timeout,document_id,datetime,analyzer_version,environment,solver
0,0,json2html,1.1.0,https://pypi.org/simple,command_error,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps json2html==1.1.0 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org","Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""'; __file__='""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-1eq417ji/json2html/pip-egg-info\n cwd: /tmp/pip-install-1eq417ji/json2html/\n Complete output (5 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-1eq417ji/json2html/setup.py"", line 17, in <module>\n long_description=open('README.md').read(),\n FileNotFoundError: [Errno 2] No such file or directory: 'README.md'\n ----------------------------------------\nERROR: Command errored out with...",1.0,"ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""'; __file__='""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-1eq417ji/json2html/pip-egg-info\n cwd: /tmp/pip-install-1eq417ji/json2html/\n Complete output (5 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-1eq417ji/json2html/setup.py"", line 17, in <module>\n long_description=open('README.md').read(),\n FileNotFoundError: [Errno 2] No such file or directory: 'README.md'\n ----------------------------------------\nERROR: Command errored out with exit status 1: python setup.py egg_info Check...",Collecting json2html==1.1.0\n Downloading https://files.pythonhosted.org/packages/70/3b/a0866f9a1ca7f3347546614770221a0061a8ea0ac5b7df053c6e55273ac8/json2html-1.1.0.tar.gz\n,60.0,solver-fedora-31-py37-00087c38,2019-12-27T17:17:58.575766,1.4.1,"{'implementation_name': 'cpython', 'implementation_version': '3.7.5', 'os_name': 'posix', 'platform_machine': 'x86_64', 'platform_python_implementation': 'CPython', 'platform_release': '3.10.0-957.21.2.el7.x86_64', 'platform_system': 'Linux', 'platform_version': '#1 SMP Tue May 28 09:26:43 UTC 2019', 'python_full_version': '3.7.5', 'python_version': '3.7', 'sys_platform': 'linux'}",solver-fedora-31-py37
1,1,guillotina,1.3.0,https://pypi.org/simple,command_error,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps guillotina==1.3.0 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org","Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-5y8hsn2x/guillotina/setup.py'""'""'; __file__='""'""'/tmp/pip-install-5y8hsn2x/guillotina/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' bdist_wheel -d /tmp/pip-wheel-l0mjgi64 --python-tag cp37\n cwd: /tmp/pip-install-5y8hsn2x/guillotina/\n Complete output (308 lines):\n running bdist_wheel\n running build\n running build_py\n creating build\n creating build/lib.linux-x86_64-3.7\n creating buil

## Split the error components from the log message

In [6]:
def split_log(log_messages):
    log_messages = log_messages.split('\n')
    return log_messages

In [7]:
error_df = pd.DataFrame()
error_df['index'] = solver_total_errors_df['index']
error_df['command'] = solver_total_errors_df['command']
error_df['package_name'] = solver_total_errors_df['package_name']
error_df['package_version'] = solver_total_errors_df['package_version']
error_df['solver'] = solver_total_errors_df['solver']

In [8]:
error_df['message'] = solver_total_errors_df['message']
error_df['split_message']= solver_total_errors_df.apply(lambda row: split_log(row.message),axis = 1)

### Example of error logs:

In [9]:
solver_total_errors_df['message'][999]

'Command exited with non-zero status code (1):     ERROR: Command errored out with exit status 1:\n     command: /home/solver/venv/bin/python3 -c \'import sys, setuptools, tokenize; sys.argv[0] = \'"\'"\'/tmp/pip-install-s_yxuv64/pyeapi/setup.py\'"\'"\'; __file__=\'"\'"\'/tmp/pip-install-s_yxuv64/pyeapi/setup.py\'"\'"\';f=getattr(tokenize, \'"\'"\'open\'"\'"\', open)(__file__);code=f.read().replace(\'"\'"\'\\r\\n\'"\'"\', \'"\'"\'\\n\'"\'"\');f.close();exec(compile(code, __file__, \'"\'"\'exec\'"\'"\'))\' egg_info --egg-base /tmp/pip-install-s_yxuv64/pyeapi/pip-egg-info\n         cwd: /tmp/pip-install-s_yxuv64/pyeapi/\n    Complete output (9 lines):\n    Traceback (most recent call last):\n      File "<string>", line 1, in <module>\n      File "/tmp/pip-install-s_yxuv64/pyeapi/setup.py", line 37, in <module>\n        from pyeapi import __version__, __author__\n      File "/tmp/pip-install-s_yxuv64/pyeapi/pyeapi/__init__.py", line 36, in <module>\n        from .client import load_config

In [10]:
error_df['split_message'][999]

['Command exited with non-zero status code (1):     ERROR: Command errored out with exit status 1:',
 '     command: /home/solver/venv/bin/python3 -c \'import sys, setuptools, tokenize; sys.argv[0] = \'"\'"\'/tmp/pip-install-s_yxuv64/pyeapi/setup.py\'"\'"\'; __file__=\'"\'"\'/tmp/pip-install-s_yxuv64/pyeapi/setup.py\'"\'"\';f=getattr(tokenize, \'"\'"\'open\'"\'"\', open)(__file__);code=f.read().replace(\'"\'"\'\\r\\n\'"\'"\', \'"\'"\'\\n\'"\'"\');f.close();exec(compile(code, __file__, \'"\'"\'exec\'"\'"\'))\' egg_info --egg-base /tmp/pip-install-s_yxuv64/pyeapi/pip-egg-info',
 '         cwd: /tmp/pip-install-s_yxuv64/pyeapi/',
 '    Complete output (9 lines):',
 '    Traceback (most recent call last):',
 '      File "<string>", line 1, in <module>',
 '      File "/tmp/pip-install-s_yxuv64/pyeapi/setup.py", line 37, in <module>',
 '        from pyeapi import __version__, __author__',
 '      File "/tmp/pip-install-s_yxuv64/pyeapi/pyeapi/__init__.py", line 36, in <module>',
 '        fro

In [11]:
solver_total_errors_df['message'][19]

'Command exited with non-zero status code (1): ERROR: Could not find a version that satisfies the requirement lupa==0.2 (from versions: 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.10, 0.11, 0.12, 0.13, 0.13.1, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 1.0b1, 1.0, 1.0.1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9)\nERROR: No matching distribution found for lupa==0.2\n'

In [12]:
error_df['split_message'][19]

['Command exited with non-zero status code (1): ERROR: Could not find a version that satisfies the requirement lupa==0.2 (from versions: 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.10, 0.11, 0.12, 0.13, 0.13.1, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 1.0b1, 1.0, 1.0.1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9)',
 'ERROR: No matching distribution found for lupa==0.2',
 '']

In [15]:
solver_total_errors_df['message'][226]

'Command exited with non-zero status code (1):   ERROR: Command errored out with exit status 1:\n   command: /home/solver/venv/bin/python3 -u -c \'import sys, setuptools, tokenize; sys.argv[0] = \'"\'"\'/tmp/pip-install-x0l8fwp4/python-bcrypt/setup.py\'"\'"\'; __file__=\'"\'"\'/tmp/pip-install-x0l8fwp4/python-bcrypt/setup.py\'"\'"\';f=getattr(tokenize, \'"\'"\'open\'"\'"\', open)(__file__);code=f.read().replace(\'"\'"\'\\r\\n\'"\'"\', \'"\'"\'\\n\'"\'"\');f.close();exec(compile(code, __file__, \'"\'"\'exec\'"\'"\'))\' bdist_wheel -d /tmp/pip-wheel-zwy9ndgr --python-tag cp37\n       cwd: /tmp/pip-install-x0l8fwp4/python-bcrypt/\n  Complete output (14 lines):\n  running bdist_wheel\n  running build\n  running build_py\n  creating build\n  creating build/lib.linux-x86_64-3.7\n  creating build/lib.linux-x86_64-3.7/bcrypt\n  copying bcrypt/__init__.py -> build/lib.linux-x86_64-3.7/bcrypt\n  running build_ext\n  building \'bcrypt._bcrypt\' extension\n  creating build/temp.linux-x86_64-3.7\n 

In [16]:
error_df['split_message'][226]

['Command exited with non-zero status code (1):   ERROR: Command errored out with exit status 1:',
 '   command: /home/solver/venv/bin/python3 -u -c \'import sys, setuptools, tokenize; sys.argv[0] = \'"\'"\'/tmp/pip-install-x0l8fwp4/python-bcrypt/setup.py\'"\'"\'; __file__=\'"\'"\'/tmp/pip-install-x0l8fwp4/python-bcrypt/setup.py\'"\'"\';f=getattr(tokenize, \'"\'"\'open\'"\'"\', open)(__file__);code=f.read().replace(\'"\'"\'\\r\\n\'"\'"\', \'"\'"\'\\n\'"\'"\');f.close();exec(compile(code, __file__, \'"\'"\'exec\'"\'"\'))\' bdist_wheel -d /tmp/pip-wheel-zwy9ndgr --python-tag cp37',
 '       cwd: /tmp/pip-install-x0l8fwp4/python-bcrypt/',
 '  Complete output (14 lines):',
 '  running bdist_wheel',
 '  running build',
 '  running build_py',
 '  creating build',
 '  creating build/lib.linux-x86_64-3.7',
 '  creating build/lib.linux-x86_64-3.7/bcrypt',
 '  copying bcrypt/__init__.py -> build/lib.linux-x86_64-3.7/bcrypt',
 '  running build_ext',
 "  building 'bcrypt._bcrypt' extension",
 '  c

In [65]:
def split_error_components(log_messages):
    ids_with_different_log_pattern = []
    Error_info, command_info, cwd, Complete_output, ERROR, specific_error, exception = {}, {}, {}, {}, {}, {}, {}
    for idx, msg in enumerate(log_messages):
        msg = msg.replace("Error:\n", "Error:")
        sentences = [x.strip() for x in msg.split('\n')]
        for id, sent in enumerate(sentences):
            if re.match(r".*WARNING.*|.*warnings.*|.*except .*|^copying.*|^checking.*", sent):
                pass
            elif re.match(r"^command.*", sent):
                if idx in command_info.keys() and sent not in command_info[idx]:
                    command_info[idx].append(sent)
                else:
                    command_info[idx] = [sent]
            elif re.match(r"^cwd.*", sent):
                if idx in cwd.keys() and sent not in cwd[idx]:
                    cwd[idx].append(sent)
                else:
                    cwd[idx] = [sent]
            elif re.match(r"^Complete output.*", sent):
                number_of_lines = re.findall(r'\d+', sent) 
                if idx in Complete_output.keys() and sent not in Complete_output[idx]:
                    pass
                else:
                    Complete_output[idx] = sentences[id:id+int(number_of_lines[0])+1]
                    for txt in Complete_output[idx]:
                        if re.match(r"^Exception.*", txt):
                            exception[idx] = [txt]
            elif re.match(r".*unable to execute.*", sent):
                specific_error[idx] = [sent]
            elif re.compile("(\w\w*Error)").findall(sent):
                if re.match('^from .* import .*', sent):
                    pass
                elif idx in specific_error.keys():
                    if sent in specific_error[idx]:
                        pass
                    if not re.match(r".*unable to execute.*", specific_error[idx][0]):
                        specific_error[idx].extend([sent])
                else:
                    specific_error[idx] = [sent]
            elif re.match(r"^Error.*", sent):
                if idx not in specific_error.keys():
                    specific_error[idx] = [sent]
            elif re.match(r"^ERROR: .*", sent):
                if idx in ERROR.keys() and sent not in ERROR[idx]:
                    if not re.match(r".*Failed.*", ERROR[idx][0]):
                        ERROR[idx].extend([sent])
                else:
                    ERROR[idx] = [sent]   
            elif re.match(r"^Command.*", sent):
                Error_info[idx] = sent
    return Error_info, command_info, cwd, Complete_output, ERROR, specific_error, exception

print(len(solver_total_errors_df['message']))

93532


In [66]:
Error_info, command_info, cwd, Complete_output, ERROR, specific_error, exception = split_error_components(solver_total_errors_df['message'])
print(len(Error_info), len(command_info), len(cwd), len(Complete_output), len(ERROR), len(specific_error), len(exception))

91522 71074 71072 71078 91346 64590 719


In [47]:
error_df['Error_info']= error_df['index'].map(Error_info)
error_df['command_info']= error_df['index'].map(command_info)
error_df['cwd']= error_df['index'].map(cwd)
error_df['Complete_output']= error_df['index'].map(Complete_output)
error_df['ERROR']= error_df['index'].map(ERROR)
error_df['Exception']= error_df['index'].map(exception)
error_df['specific_error']= error_df['index'].map(specific_error)

In [48]:
error_df.head(10)

,index,command,package_name,package_version,solver,message,split_message,Error_info,command_info,cwd,Complete_output,ERROR,Exception,specific_error,clustering_data,tokenized_clustering_data
0,0,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps json2html==1.1.0 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org",json2html,1.1.0,solver-fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""'; __file__='""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-1eq417ji/json2html/pip-egg-info\n cwd: /tmp/pip-install-1eq417ji/json2html/\n Complete output (5 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-1eq417ji/json2html/setup.py"", line 17, in <module>\n long_description=open('README.md').read(),\n FileNotFoundError: [Errno 2] No such file or directory: 'README.md'\n ----------------------------------------\nERROR: Command errored out with...","[Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:, command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""'; __file__='""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-1eq417ji/json2html/pip-egg-info, cwd: /tmp/pip-install-1eq417ji/json2html/, Complete output (5 lines):, Traceback (most recent call last):, File ""<string>"", line 1, in <module>, File ""/tmp/pip-install-1eq417ji/json2html/setup.py"", line 17, in <module>, long_description=open('README.md').read(),, FileNotFoundError: [Errno 2] No such file or directory: 'README.md', ----------------------------------------, ERROR: Command errored out wit...",Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:,"[command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""'; __file__='""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-1eq417ji/json2html/pip-egg-info]",[cwd: /tmp/pip-install-1eq417ji/json2html/],"[Complete output (5 lines):, Traceback (most recent call last):, File ""<string>"", line 1, in <module>, File ""/tmp/pip-install-1eq417ji/json2html/setup.py"", line 17, in <module>, long_description=open('README.md').read(),, FileNotFoundError: [Errno 2] No such file or directory: 'README.md']",[ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.],NaN,[FileNotFoundError: [Errno 2] No such file or directory: 'README.md'],FileNotFoundError,[FileNotFoundError]
1,1,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps guillotina==1.3.0 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org",guillotina,1.3.0,solver-fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-5y8hsn2x/guillotina/setup.py'""'""'; __file__='""'""'/tmp/pip-install-5y8hsn2x

## Prepare the data for clustering

Look for data in the 'specific_error' column of the dataframe. If not present, look at 'ERROR' column or 'Error_info' column. If all the extracted columns are empty, take the error data from 'message' column. 

In [50]:
clustering_data = {}
for idx, error in enumerate(error_df['specific_error']):
    if type(error) != list:
        #if type(error_df.iloc[idx]['Exception']) == list:
            #clustering_data[idx] = error_df.iloc[idx]['Exception']
        if pd.isnull(error) and  type(error_df.iloc[idx]['ERROR']) == list:
            r = re.compile(".*Failed.*|.*CUDA.*")
            match = list(filter(r.match, error_df.iloc[idx]['ERROR']))    
            if len(error_df.iloc[idx]['ERROR']) == 1 or not match:
                clustering_data[idx] = [error_df.iloc[idx]['ERROR'][0]]
            else:
                clustering_data[idx] = match
        elif type(error_df.iloc[idx]['ERROR']) != list:
            if pd.isnull(error_df.iloc[idx]['ERROR']):
                clustering_data[idx] = [error_df.iloc[idx]['Error_info']]
                if type(error_df.iloc[idx]['Error_info']) != list:
                    if pd.isnull(error_df.iloc[idx]['Error_info']):
                        clustering_data[idx] = [error_df.iloc[idx]['message']]        
    else:
        # Extracting just the words with 'Error' as suffix to it and handling some edge cases. 
        r = re.compile("(\w\w*Error)")     
        string = ' '.join(error)
        match = re.findall(r, string)
        if match:
            if len(match) == 1:
                clustering_data[idx] = match
            else:
                if 'SyntaxError' in match:
                    for err in match:
                        if re.match(rf'.*{err},|.*{err}\),', string):
                            clustering_data[idx] = ['SyntaxError']
                            break
                    if idx not in clustering_data.keys():
                        clustering_data[idx] = list(set(match))
                elif {'OSError', 'EnvironmentError'} == set(match):
                    clustering_data[idx] = ['OSError']
                elif {'Error_GetLastError', 'AttributeError'} == set(match):
                    clustering_data[idx] = ['AttributeError']
                elif {'GrabNetworkError', 'GrabError', 'ModuleNotFoundError'} == set(match):
                    clustering_data[idx] = ['ModuleNotFoundError']
                elif {'ReadTimeoutError', 'EnvironmentError'} == set(match):
                    clustering_data[idx] = ['ReadTimeoutError']
                elif {'NewConnectionError', 'EnvironmentError'} == set(match):
                    clustering_data[idx] = ['NewConnectionError']
                else:
                    clustering_data[idx] = list(set(match))
        else:
            clustering_data[idx] = error
    if clustering_data[idx] == ['Command exited with non-zero status code (-9):']:
        clustering_data[idx] = ['Error Info not Available']
    string = ' '.join(clustering_data[idx])
    if re.match(".*make: .*|.*CMake .*|.*zError.*|.*SWIG_Python_TypeError.*|.*CatchlibLZMAError.*|.*PyExc_MemoryError.*|.*ViZDoomError.*|.*PyExc_KeyError.*|.*icsneoGetError.*|.*LapackSrcNotFoundError.*|.*PyFrame_FastToLocalsWithError.*", string):
    #if re.match(".*zError.*|.*SWIG_Python_TypeError.*|.*CatchlibLZMAError.*|.*PyExc_KeyError.*|.*PyFrame_FastToLocalsWithError.*", clustering_data[idx][0]):
        clustering_data[idx] = error_df.iloc[idx]['ERROR']

In [51]:
len(clustering_data)

93532

## Cleaning clustering data

In [52]:
_line_number = r'(at line[:]*\s*\d+)'
_url = r'(http[s]|root|srm|file)*:(//|/)(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
_filepath = "(/[a-zA-Z\./]*[\s]?)"
path_regex = re.compile(r'(\b\w+://)\S+(?=\s)')
file_regex = re.compile(r'(\b[f|F]ile( exists)?:?\s?)/\S+(?=\s)')
py_regex = re.compile(r'/?\b[-./_a-zA-Z0-9]+\.py\b')
long_regex = re.compile(r'[-/_a-zA-Z0-9]{25,}')

In [53]:
def remove_whitespaces(sentence):
    return " ".join(sentence.split())

def substitute_path(string):
    string = path_regex.sub(r'\1', string)
    string = py_regex.sub(r' ', string)
    string = file_regex.sub(r'\1', string)
    string = long_regex.sub(r'', string)
    return string

def cleaner(log_messages):
    clean_log = {}
    for key in log_messages:
        for item in log_messages[key]:
            item = re.sub(_line_number, "at line *", item)
            item = re.sub(_url, " ", item)
            item = re.sub(_filepath, " ", item)
            item = substitute_path(item)
            if key in clean_log.keys() and item not in clean_log[key]:
                clean_log[key] = clean_log[key] + ' ' + remove_whitespaces(item)
            else:
                clean_log[key] = remove_whitespaces(item)
    return clean_log

In [54]:
clustering_data = cleaner(clustering_data)

In [55]:
len(clustering_data)

93532

## Tokenization 

In [56]:
stemmer = nltk.PorterStemmer()
stop = punctuation + "``" + "''" + '""' + "/"
table = str.maketrans(stop, ' '*len(stop))

In [57]:
def tokenization(log_messages):
    tokenized = []
    for key, item in log_messages.items():
        item = item.replace(' Error','Error').strip()
        item = item.replace('Errno','').strip()
        item = item.replace('Error :','Error:').strip()
        item = item.replace('Exception:','').strip()
        item = item.replace('Exception','').strip()
        item = item.replace('\'','').strip()
        item = item.replace('=',' ').strip()
        if error_df['package_name'][key] in item.split():
            item = item.replace(error_df['package_name'][key],'').strip()
        if "interpreter:" in item.split():
            item = item.split("interpreter",1)[1]
        if " Please" in item:
            item = item.split("Please",1)[0]
        if "for"  in item.split():
            item = item.split("for",1)[0] 
        if "in" in item.split():
            if "in an" in item:
                item = item.split("in an",1)[1]
            else:
                item = item.split("in",1)[0]
        if "on" in item.split():
            item = re.split(" on ",item)[0] 
        if re.match(r"^ERROR:.* |^Error:.* |^error:.*", item):
            item = re.split("^ERROR: |^error: |^Error: |^error:Error:",item)[1].split('.')[0]
        if item.strip() == '':
            item = error_df.iloc[idx]['ERROR'][0].translate(table).strip()
        if "Command" in item:
            if "ERROR:" in item:
                item = item.split("ERROR:",1)[1] 
            if re.match(r".*Command errored out with exit status.*", item):
                item = "Check the logs"
        if "not found" in item:
            words = [stemmer.stem(word) for word in item.split()] 
            item = ''
            for word in words:
                item += word[0].upper() + word[1:]
            item += "Error"
        if ":" in item and item[0] != ":":
            item = item.split(":",1)[0] 
        if "," in item:
            item = item.split(",",1)[0] 
        if "JAVA_HOME" in item:
            item = 'JAVA HOME not set to a path containing the JDK'
        if re.match("(\w\w* error)", item):
            item = item.split()[0] + "Error"
        if "XML" in item:
            item = "cannot get pre-processor and compiler flags"
        item = item.translate(table).strip()
        tokenized.append(TreebankWordTokenizer().tokenize(item))
    cleaned_tokens = []
    for id, row in enumerate(tokenized):
        cleaned_tokens.append(list(filter(None, [i
                                                 for i in row 
                                                 if i != error_df['package_name'][id]
                                                 and not i.lower().isnumeric()])))
    return cleaned_tokens

In [58]:
tokenized_clustering_data = tokenization(clustering_data)

In [59]:
tokenized_clustering_data

[['FileNotFoundError'],
 ['unable', 'to', 'execute', 'gcc'],
 ['unable', 'to', 'execute', 'gcc'],
 ['NameError'],
 ['No', 'matching', 'distribution', 'found'],
 ['SyntaxError'],
 ['unable', 'to', 'execute', 'gcc'],
 ['ModuleNotFoundError'],
 ['FileNotFoundError'],
 ['FileNotFoundError'],
 ['TypeError'],
 ['FileNotFoundError'],
 ['No', 'matching', 'distribution', 'found'],
 ['SyntaxError'],
 ['FileNotFoundError'],
 ['SyntaxError'],
 ['NameError'],
 ['ModuleNotFoundError'],
 ['Check', 'the', 'logs'],
 ['No', 'matching', 'distribution', 'found'],
 ['SyntaxError'],
 ['SyntaxError'],
 ['SyntaxError'],
 ['SyntaxError'],
 ['Failed', 'building', 'wheel'],
 ['unable', 'to', 'execute', 'gcc'],
 ['unable', 'to', 'execute', 'gcc'],
 ['FileNotFoundError'],
 ['SyntaxError'],
 ['SyntaxError'],
 ['SyntaxError'],
 ['ImportError'],
 ['SyntaxError'],
 ['ImportError', 'HTTPError'],
 ['ModuleNotFoundError'],
 ['ModuleNotFoundError'],
 ['No', 'matching', 'distribution', 'found'],
 ['SyntaxError'],
 ['FileNo

## Save the cleaned data for clustering

In [60]:
error_df['clustering_data'] = error_df['index'].map(clustering_data)

In [61]:
error_df['tokenized_clustering_data'] = tokenized_clustering_data

In [62]:
error_df.head(20)

,index,command,package_name,package_version,solver,message,split_message,Error_info,command_info,cwd,Complete_output,ERROR,Exception,specific_error,clustering_data,tokenized_clustering_data
0,0,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps json2html==1.1.0 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org",json2html,1.1.0,solver-fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""'; __file__='""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-1eq417ji/json2html/pip-egg-info\n cwd: /tmp/pip-install-1eq417ji/json2html/\n Complete output (5 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-1eq417ji/json2html/setup.py"", line 17, in <module>\n long_description=open('README.md').read(),\n FileNotFoundError: [Errno 2] No such file or directory: 'README.md'\n ----------------------------------------\nERROR: Command errored out with...","[Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:, command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""'; __file__='""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-1eq417ji/json2html/pip-egg-info, cwd: /tmp/pip-install-1eq417ji/json2html/, Complete output (5 lines):, Traceback (most recent call last):, File ""<string>"", line 1, in <module>, File ""/tmp/pip-install-1eq417ji/json2html/setup.py"", line 17, in <module>, long_description=open('README.md').read(),, FileNotFoundError: [Errno 2] No such file or directory: 'README.md', ----------------------------------------, ERROR: Command errored out wit...",Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:,"[command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""'; __file__='""'""'/tmp/pip-install-1eq417ji/json2html/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-1eq417ji/json2html/pip-egg-info]",[cwd: /tmp/pip-install-1eq417ji/json2html/],"[Complete output (5 lines):, Traceback (most recent call last):, File ""<string>"", line 1, in <module>, File ""/tmp/pip-install-1eq417ji/json2html/setup.py"", line 17, in <module>, long_description=open('README.md').read(),, FileNotFoundError: [Errno 2] No such file or directory: 'README.md']",[ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.],NaN,[FileNotFoundError: [Errno 2] No such file or directory: 'README.md'],FileNotFoundError,[FileNotFoundError]
1,1,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps guillotina==1.3.0 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org",guillotina,1.3.0,solver-fedora-31-py37,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-5y8hsn2x/guillotina/setup.py'""'""'; __file__='""'""'/tmp/pip-install-5y8hsn2x

In [63]:
#with open("solver-errors-clean-clustering-data.txt", "wb") as fp: 
 #   pickle.dump(error_df['tokenized_clustering_data'], fp)

In [64]:
error_df.to_csv('error-clean-data.csv', index=False)